# Notebook for final pfoject of IBM's professional Data Science specialization

In [1]:
import pandas as pd
import numpy as np
from pandas.io.html import read_html

In [2]:
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

In [3]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
DFs = read_html(page, match='Borough', header=0) # a list of dataframes
df = DFs[0]#getting the dataframe frm the list

### Exploring the data:

In [7]:
df.describe()

,Postcode,Borough,Neighbourhood
count,288,288,288
unique,180,12,209
top,M9V,Not assigned,Not assigned
freq,8,77,78


In [18]:
print('not assigned boroughs: ', df.loc[df.Borough == 'Not assigned', 'Borough'].count())
print('not assigned neighbourhoods: ', df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'].count())

not assigned boroughs:  77
not assigned neighbourhoods:  78


### Ignore cells with a borough that is Not assigned:

In [19]:
df.Borough.replace('Not assigned', np.nan, inplace=True)

In [21]:
df.dropna(subset=['Borough'], inplace=True)
print('not assigned boroughs: ', df.loc[df.Borough == 'Not assigned', 'Borough'].count())

not assigned boroughs:  0


### grouping repeated neighbourhoods/postcode:

In [22]:
df.set_index(['Postcode', 'Borough'], inplace=True)

In [23]:
df.groupby(level=['Postcode','Borough']).agg(','.join)

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


### processing not assigned neighbourhoods:

In [27]:
df.reset_index(inplace=True)
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [29]:
print('not assigned neighbourhoods: ', df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'].count())


not assigned neighbourhoods:  1


In [30]:
df.loc[df.Neighbourhood == 'Not assigned', 'Borough']

6    Queen's Park
Name: Borough, dtype: object

In [32]:
df.at[85, 'Neighbourhood'] = 'Queen\'s park'

In [34]:
df.shape

(211, 3)